## 1. Execute full pipeline

In [2]:
import random
import time

from keras_tuner import RandomSearch

import numpy as np

import pandas as pd

import tensorflow as tf

from config.constants import (
    FORECAST_HORIZON, NB_TRIALS,
    OBSERVATION_WINDOW, SEED, TRAIN_PERC
)

from src.change_point_detector import ChangePointCostFunction, ChangePointMethod, get_change_point_detector
from src.dataset import read_dataset, split_X_y, split_train_test
from src.forecaster import InternalForecaster, TimeSeriesHyperModel
from src.scaler import Scaler
from src.utils import get_error_results

tf.get_logger().setLevel('ERROR')

np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)


In [3]:
print("Available devices:", tf.config.list_physical_devices())
print("Is GPU available?", tf.config.list_physical_devices('GPU'))


Available devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Is GPU available? [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
if tf.config.list_physical_devices("GPU"):
    try:
        tf.config.set_visible_devices([], "GPU")  # Hide traditional GPUs if any (like external ones)
        tf.config.experimental.set_memory_growth(tf.config.list_physical_devices("GPU")[0], True)
        print("✅ TensorFlow is using Apple GPU via MPS")
    except Exception as e:
        print("⚠️ Error setting MPS GPU:", e)
else:
    print("❌ No GPU found, running on CPU")

✅ TensorFlow is using Apple GPU via MPS


In [5]:
timestamp = 'validate_pipeline'
dataset_domain_argv = 'UCI'
dataset_argv = 'APPLIANCES_ENERGY'
change_point_method_argv = 'Window'
change_point_cost_function_argv = 'L1'

In [6]:
execution_id = f"{timestamp}_{dataset_domain_argv}_{dataset_argv}_{change_point_method_argv}_{change_point_cost_function_argv}_{SEED}"
change_point_method = ChangePointMethod.from_str(change_point_method_argv)
change_point_cost_function = ChangePointCostFunction.from_str(change_point_cost_function_argv)
change_point_approach = f"{change_point_method.value.title()} {change_point_cost_function.value.title()}"

In [7]:
print(f"[Step 1] Reading dataset {dataset_argv} from {dataset_domain_argv}")
df, variables = read_dataset(dataset_domain_argv, dataset_argv)
print(f"Variables: {variables}")

[Step 1] Reading dataset APPLIANCES_ENERGY from UCI
Variables: ['T_out', 'Press_mm_hg', 'RH_out', 'Windspeed', 'Visibility', 'Tdewpoint']


In [8]:
print("[Step 2] Splitting data into train_val and test")
train_val, test = split_train_test(df)
report = {
    'execution_id': execution_id,
    'timestamp': timestamp,
    'change_point_method': change_point_method.value,
    'change_point_cost_function': change_point_cost_function.value,
    'change_point_approach': change_point_approach,
    'seed': SEED,
    'observation_window': OBSERVATION_WINDOW,
    'train_perc': TRAIN_PERC,
    'nb_trials': NB_TRIALS,
    'dataset_domain': dataset_domain_argv,
    'dataset': dataset_argv,
    'variables': variables,
    'dataset_shape': df.shape,
    'train_val_shape': train_val.shape,
    'test_shape': test.shape,
}

[Step 2] Splitting data into train_val and test


In [9]:
print(f"[Step 3] Detecting cut point ({change_point_approach})")
start_time = time.time()
change_point_detector = get_change_point_detector(change_point_method, change_point_cost_function)
change_point, change_point_perc = change_point_detector.find_change_point(train_val, variables)
end_time = time.time()
detect_change_point_duration = end_time - start_time
print(f"Change point: {change_point}, Change point percentage: {change_point_perc}")
report.update({
    'detect_change_point_duration': detect_change_point_duration,
    'change_point': str(change_point),
    'change_point_perc': change_point_perc
})

[Step 3] Detecting cut point (Window L1)
Change point: 8460, Change point percentage: 53.5850012667849


In [10]:
print("[Step 4] Reducing train_val based on change point")
start_time = time.time()
reduced_train_val = change_point_detector.apply_change_point(train_val, change_point)
end_time = time.time()
apply_change_point_duration = end_time - start_time
report.update({
    'apply_change_point_duration': apply_change_point_duration,
    'reduced_train_val.shape': reduced_train_val.shape,
})

[Step 4] Reducing train_val based on change point


In [11]:
print("[Step 5] Splitting train_val into train and val")
reduced_train, reduced_val = split_train_test(reduced_train_val)
report.update({
    'reduced_train.shape': reduced_train.shape,
    'reduced_val.shape': reduced_val.shape,
})

[Step 5] Splitting train_val into train and val


In [12]:
print("[Step 6] Fitting scaler on train and applying on train and val")
start_time = time.time()
scaler = Scaler(variables)
scaled_reduced_train = scaler.fit_scale(reduced_train)
scaled_reduced_val = scaler.scale(reduced_val)
end_time = time.time()
fit_apply_scaler_train_val_duration = end_time - start_time
report.update({
    'fit_apply_scaler_train_val_duration': fit_apply_scaler_train_val_duration,
})

[Step 6] Fitting scaler on train and applying on train and val


In [13]:
print("[Step 7] Splitting train and val into X and y")
X_reduced_scaled_train, y_reduced_scaled_train = split_X_y(scaled_reduced_train)
X_reduced_scaled_val, y_reduced_scaled_val = split_X_y(scaled_reduced_val)
report.update({
    'X_reduced_scaled_train.shape': X_reduced_scaled_train.shape,
    'y_reduced_scaled_train.shape': y_reduced_scaled_train.shape,
    'X_reduced_scaled_val.shape': X_reduced_scaled_val.shape,
    'y_reduced_scaled_val.shape': y_reduced_scaled_val.shape,
})

[Step 7] Splitting train and val into X and y


In [14]:
y_reduced_scaled_train[0]

array([[ 0.42508344,  0.34656565, -1.55042352, -1.21398052, -1.10125741,
        -0.90743084],
       [ 0.43823785,  0.34907256, -1.60052507, -1.21398052, -1.16634056,
        -0.94966489],
       [ 0.45139227,  0.35157947, -1.65062661, -1.21398052, -1.2314237 ,
        -0.99189893],
       [ 0.46454668,  0.35408636, -1.70072815, -1.21398052, -1.29650684,
        -1.03061347],
       [ 0.47770109,  0.35659326, -1.75082969, -1.21398052, -1.36158999,
        -1.07284751],
       [ 0.4908555 ,  0.36662088, -1.75082969, -1.21398052, -1.34857336,
        -1.04926683],
       [ 0.50400991,  0.37664847, -1.75082969, -1.21398052, -1.33555673,
        -1.02709396]])

In [15]:
y_reduced_scaled_train[1]

array([[ 0.43823785,  0.34907256, -1.60052507, -1.21398052, -1.16634056,
        -0.94966489],
       [ 0.45139227,  0.35157947, -1.65062661, -1.21398052, -1.2314237 ,
        -0.99189893],
       [ 0.46454668,  0.35408636, -1.70072815, -1.21398052, -1.29650684,
        -1.03061347],
       [ 0.47770109,  0.35659326, -1.75082969, -1.21398052, -1.36158999,
        -1.07284751],
       [ 0.4908555 ,  0.36662088, -1.75082969, -1.21398052, -1.34857336,
        -1.04926683],
       [ 0.50400991,  0.37664847, -1.75082969, -1.21398052, -1.33555673,
        -1.02709396],
       [ 0.51716432,  0.38667608, -1.75082969, -1.21398052, -1.3225401 ,
        -1.00245744]])

In [16]:
y_reduced_scaled_train[2]

array([[ 0.45139227,  0.35157947, -1.65062661, -1.21398052, -1.2314237 ,
        -0.99189893],
       [ 0.46454668,  0.35408636, -1.70072815, -1.21398052, -1.29650684,
        -1.03061347],
       [ 0.47770109,  0.35659326, -1.75082969, -1.21398052, -1.36158999,
        -1.07284751],
       [ 0.4908555 ,  0.36662088, -1.75082969, -1.21398052, -1.34857336,
        -1.04926683],
       [ 0.50400991,  0.37664847, -1.75082969, -1.21398052, -1.33555673,
        -1.02709396],
       [ 0.51716432,  0.38667608, -1.75082969, -1.21398052, -1.3225401 ,
        -1.00245744],
       [ 0.53031874,  0.39670369, -1.75082969, -1.21398052, -1.30952347,
        -0.97782091]])

In [ ]:
print("[Step 8] Running HPO and NAS")
n_variables = len(variables)
forecaster_hypermodel = TimeSeriesHyperModel(
    n_variables=n_variables
)
forecaster_tuner = RandomSearch(
    forecaster_hypermodel,
    objective='val_loss',
    max_trials=3,
    executions_per_trial=1,
    directory=f"outputs/tuner/delete_me",
    project_name="delete_me",
    seed=SEED,
    overwrite=True,
    distribution_strategy=tf.distribute.MirroredStrategy()
)
start_time = time.time()
forecaster_tuner.search(
    X_reduced_scaled_train,
    y_reduced_scaled_train,
    validation_data=(X_reduced_scaled_val, y_reduced_scaled_val),
    shuffle=False,
)
end_time = time.time()
tuner_duration = end_time - start_time
report.update({
    'tuner_duration': tuner_duration
})

[Step 8] Running HPO and NAS

Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
2                 |2                 |num_layers
64                |64                |units_0
0.001             |0.001             |learning_rate

Epoch 1/25


2025-03-08 20:29:41.252273: W tensorflow/core/framework/dataset.cc:959] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


In [ ]:
print("[Step 9] Retrieving best model")
best_trial = forecaster_tuner.oracle.get_best_trials(num_trials=1)[0]
best_forecaster_model = forecaster_tuner.get_best_models(num_models=1)[0]
print(f"Trial ID: {best_trial.trial_id}")
print(f"Hyperparameters: {best_trial.hyperparameters.values}")
print(f"Score: {best_trial.score}")
print("-" * 40)
best_forecaster_model.summary()
best_forecaster_model = InternalForecaster(
    best_forecaster_model,
    len(variables),
    best_trial.hyperparameters.values['batch_size'],
    best_trial.hyperparameters.values['epochs'],
)
report.update({
    'best_trial_id': best_trial.trial_id,
    'best_trial_hyperparameters': best_trial.hyperparameters.values,
    'best_trial_score': best_trial.score,
    'best_forecaster_model': best_forecaster_model.summary(),
})

In [ ]:
print("[Step 10] Fitting scaler on train_val and applying on train_val and test")
start_time = time.time()
scaler = Scaler(variables)
scaled_reduced_train_val = scaler.fit_scale(reduced_train_val)
scaled_test = scaler.scale(test)
end_time = time.time()
fit_apply_scaler_train_val_test_duration = end_time - start_time
report.update({
    'fit_apply_scaler_train_val_test_duration': fit_apply_scaler_train_val_test_duration,
})

In [ ]:
print("[Step 11] Splitting train_val and test into X and y")
X_reduced_scaled_train_val, y_reduced_scaled_train_val = split_X_y(scaled_reduced_train_val)
X_scaled_test, y_scaled_test = split_X_y(scaled_test)
report.update({
    'X_reduced_scaled_train_val.shape': X_reduced_scaled_train_val.shape,
    'y_reduced_scaled_train_val.shape': y_reduced_scaled_train_val.shape,
    'X_scaled_test.shape': X_scaled_test.shape,
    'y_scaled_test.shape': y_scaled_test.shape,
})

In [ ]:
print("[Step 12] Retraining best model")
start_time = time.time()
best_forecaster_model.fit(
    X_reduced_scaled_train_val,
    y_reduced_scaled_train_val,
    shuffle=False
)
end_time = time.time()
retrain_duration = end_time - start_time
report.update({
    'retrain_duration': retrain_duration,
})

In [ ]:
print("[Step 13] Forecasting for test")
start_time = time.time()
y_scaled_pred = best_forecaster_model.forecast(X_scaled_test)
y_scaled_test_flat = y_scaled_test.reshape(-1, n_variables)
y_scaled_pred_flat = y_scaled_pred.reshape(-1, n_variables)
end_time = time.time()
forecasting_test_duration = end_time - start_time
report.update({
    'forecasting_test_duration': forecasting_test_duration,
})

In [ ]:
y_scaled_test_flat

In [ ]:
y_scaled_pred_flat

In [ ]:
print("[Step 14] Descaling data")
start_time = time.time()
y_test = scaler.descale(pd.DataFrame(y_scaled_test_flat, columns=variables))
y_pred = scaler.descale(pd.DataFrame(y_scaled_pred_flat, columns=variables))
end_time = time.time()
descaling_duration = end_time - start_time
report.update({
    'descaling_duration': descaling_duration,
})

In [ ]:
y_test

In [ ]:
y_pred

In [ ]:
print("[Step 15] Calculating evaluation metrics")
total_duration = sum(value for key, value in report.items() if key.endswith('_duration'))
error_results = get_error_results(y_test, y_pred, variables)
print(f"Obtained error results: {error_results}")
report.update({
    'total_duration': total_duration,
    'error_results': error_results,
})

In [ ]:
report

## 2. What would be the error if we predicted the average values for all variables (Dummy Forecaster)?

In [ ]:
X_train_val, y_train_val = split_X_y(train_val)
X_test, y_test = split_X_y(test)

In [ ]:
train_val_targets_flat = pd.DataFrame(y_train_val.reshape(-1, len(variables)), columns=variables)
avg_values = train_val_targets_flat.mean(axis=0).to_numpy()

In [ ]:
n_test = y_test.shape[0]
dummy_pred = np.tile(avg_values, (n_test, FORECAST_HORIZON, 1))

dummy_pred_flat = dummy_pred.reshape(-1, n_variables)
y_test_flat = pd.DataFrame(y_test.reshape(-1, n_variables), columns=variables)

In [ ]:
dummy_error_results = get_error_results(y_test_flat, dummy_pred_flat, variables)
print(f"Error metrics for Dummy Forecaster (predicting average values): \n{dummy_error_results}")

In [ ]:
df_comparison = pd.DataFrame({
    "Trained Model": pd.Series(error_results),
    "Dummy Forecaster": pd.Series(dummy_error_results)
})

df_comparison = df_comparison.round(5)
df_comparison